In [13]:
# ================================================================
# 📦 Imports & Environment Setup
# ================================================================
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

# Load .env variables
load_dotenv()
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
print(SERPER_API_KEY)
# Set default OpenAI model for tools that need it
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

# ---------------------------------------------------------------
# 🤖 Configure LLM (Ollama with Qwen2.5 model)
# ---------------------------------------------------------------
llm = LLM(model="ollama/qwen2.5")


# ================================================================
# 🔍 Scraper Agent — Uses Serper + ScrapeWebsiteTool
# ================================================================
scraper_agent = Agent(
    role="Web Scraper",
    goal=(
        "Conduct in-depth, real-time analysis on the topic and get the "
        "latest news and market updates."
    ),
    backstory=(
        "As the Lead Analyst, you specialize in gathering the latest news and "
        "market trends, ensuring the content is ahead of the curve and provides "
        "relevant insights."
    ),
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=llm,
    allow_delegation=False
)

scrape_blog = Task(
    description="""
    Analyze market data and trends related to {topic}, with a focus on uncovering
    patterns, opportunities, and risks that impact the market landscape.

    Pay close attention to the latest advancements in {topic} that influence
    market movements, investor sentiment, and economic signals.
    """,
    expected_output="""
    A comprehensive analysis report highlighting:
    - Key market trends related to {topic}
    - Actionable insights
    - Recommendations for content creation
    """,
    agent=scraper_agent
)


# ================================================================
# 📝 Draft Analyzer Agent — Reads & Decomposes the Markdown
# ================================================================
draft_analyzer = Agent(
    role="Draft Analyzer",
    goal="Analyze draft content and identify key ideas, structure, and technical concepts.",
    backstory=(
        "You are a seasoned technical writer. You excel at mapping ideas, "
        "identifying section-level structure, and extracting key technical concepts."
    ),
    llm=llm
)

analyze_draft = Task(
    description="""
    Analyze the markdown content to create a developer-focused technical overview.
    
    Steps:
    1. Map out the core idea discussed in the content.
    2. Identify key sections and describe what each section covers.
    3. Extract any important technical concepts, code examples, or system insights.

    Focus on details necessary for a comprehensive understanding of the blog.
    """,
    expected_output="""
    A technical analysis containing:
    - Title and core concept/idea
    - Key technical sections & their main points
    - Important code examples or engineering concepts
    - Key takeaways for developers
    - Relevant media URLs that match sections (to be used for tweets)
    """,
    agent=draft_analyzer
)


# ================================================================
# 🐦 X Thread Planner Agent
# ================================================================
x_analyzer = Agent(
    role="X Thread Planner",
    goal="Create an X/Twitter thread plan based on the provided draft analysis.",
    backstory="""
    You convert long-form content into viral, bite-sized, high-retention X threads.
    You understand pacing, hooks, storytelling, and technical clarity.
    """,
    llm=llm
)

# Create the file if it does not exist
if not os.path.exists("example_threads.txt"):
    with open("example_threads.txt", "w", encoding="utf-8") as f:
        f.write("")   # write empty content or default template
        
with open("example_threads.txt", "r", encoding="utf-8") as f:
    example_threads = f.read()

write_x = Task(
    description=f"""
    Develop an engaging Twitter thread based on the provided draft analysis.

    Requirements:
    - Strong hook tweet (<10 words)
    - Logical flow from basic → advanced concepts
    - Include code snippets or technical highlights
    - Attach relevant media URLs to each tweet
    - Follow the exact writing style from these examples:
      {example_threads}

    The thread must cover:
    - The main problem being solved
    - Key technical innovations
    - Implementation insights
    - Real-world impact
    - Clear final call to action
    """,
    expected_output="""
    A Twitter thread (list of tweets), each containing:
    - content
    - media URLs (when possible)
    - is_hook: true/false
    """,
    agent=x_analyzer
)


# ================================================================
# 🔗 LinkedIn Post Generator Agent
# ================================================================
linkedin_analyzer = Agent(
    role="LinkedIn Post Planner",
    goal="Create an engaging LinkedIn post using the draft analysis.",
    backstory="""
    You specialize in creating high-impact LinkedIn content that is authoritative
    yet accessible. You blend technical clarity with professional storytelling.
    """,
    llm=llm
)

# Create the file if it does not exist
if not os.path.exists("example_linkedin.txt"):
    with open("example_linkedin.txt", "w", encoding="utf-8") as f:
        f.write("")   # write empty content or default template

with open("example_linkedin.txt", "r", encoding="utf-8") as f:
    example_linkedin = f.read()

write_linkedin = Task(
    description=f"""
    Develop a comprehensive LinkedIn post using the draft analysis.
    Follow the writing style provided in:
    {example_linkedin}

    Requirements:
    - Strong attention-grabbing opening
    - Structured narrative with section clarity
    - Technical depth + professional tone
    - Use 3–5 strategic hashtags
    - Include one main blog URL at the end
    """,
    expected_output="""
    A LinkedIn post plan containing:
    - content
    - main blog URL associated with the post
    """,
    agent=linkedin_analyzer
)


76ecc6fc8c755d1561f41b669eb313b1b45dfe84


In [ ]:
from crewai import Crew

crew = Crew(
    agents=[scraper_agent,draft_analyzer, x_analyzer, linkedin_analyzer],
    tasks=[scrape_blog, analyze_draft, write_x, write_linkedin],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"Agentic AI","example_threads":example_threads,"example_linkedin":example_linkedin})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cd7cec9c-40a1-4af4-93a1-d5b5a4936dfa                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Scraper                                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze market data and trends related to Agentic AI, with a focus on uncovering                           │
│      patterns, opportunities, and risks that impact the market landscape.                                       │
│                                                                                                                 │
│      Pay close attention to the latest advancements in Agentic AI that influence                                │
│      market movements, investor sentiment, and economic signals.                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Scraper                                                                                             │
│                                                                                                                 │
│  Thought: Thought: To conduct an in-depth analysis of market data and trends related to Agentic AI, I need to   │
│  gather the latest news and updates from reliable sources. This will help me identify key market trends,        │
│  actionable insights, and recommendations for content creation.                                                 │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "Agentic AI market trends 2023"                                                              │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Scraper                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Agentic AI Market Analysis Report                                                                            │
│                                                                                                                 │
│  ## Key Market Trends Related to Agentic AI                                                                     │
│                                                                                                                 │
│  ### 1. **Market Growth and Expansion**                                                                         │
│     - The global Agentic AI market is expected to grow significantly, with projections indicating a CAGR of     │
│  around 43-45% over the next few years.                                                                         │
│     - Major regions such as North America, Europe, and Asia-Pacific are anticipated to drive this growth due    │
│  to increasing adoption in various industries.                                                                  │
│                                                                                                                 │
│  ### 2. **Technological Advancements**                                                                          │
│     - Innovations in AI algorithms and machine learning capabilities are driving the development of more        │
│  sophisticated Agentic AI solutions.                                                                            │
│     - Companies are focusing on enhancing the natural language processing (NLP) abilities of their agents,      │
│  enabling them to interact more effectively with users.                                                         │
│                                                                                                                 │
│  ### 3. **Market Segmentation**                                                                                 │
│     - The market is segmented based on various factors such as application (customer service, healthcare,       │
│  finance), deployment model (on-premises, cloud-based), and end-user industry.                                  │
│     - Healthcare and financial services are expected to be major drivers of demand due to the increasing need   │
│  for automation in these sectors.                                                                               │
│                                                                                                                 │
│  ### 4. **Supply Exceeding Demand**                                                                             │
│     - According to Gartner's report, the current supply of Agentic AI models, platforms, and products far       │
│  exceeds demand, leading to market consolidation and potential correction.                                      │
│     - This excess supply is driving competition among players, which could result in price wars or mergers and  │
│  acquisitions.                                                                                                  │
│                                                                                                                 │
│  ### 5. **Regulatory Landscape**                                                                                │
│     - The regulatory environment for Agentic AI is evol

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1d81095a-a901-4d11-bb0e-57bfbf51eb5a                                                                     │
│  Agent: Web Scraper                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Draft Analyzer                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze the markdown content to create a developer-focused technical overview.                             │
│                                                                                                                 │
│      Steps:                                                                                                     │
│      1. Map out the core idea discussed in the content.                                                         │
│      2. Identify key sections and describe what each section covers.                                            │
│      3. Extract any important technical concepts, code examples, or system insights.                            │
│                                                                                                                 │
│      Focus on details necessary for a comprehensive understanding of the blog.                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()